In [ ]:
import os
from dotenv import load_dotenv

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

In [ ]:
# Instala o python-dotenv se não estiver instalado
try:
    import dotenv
except ImportError:
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "python-dotenv"])

In [ ]:
from apify_client import ApifyClient

# Initialize the ApifyClient with your API token from environment variable
client = ApifyClient(os.getenv("APIFY_API_TOKEN"))

# Prepare the Actor input
run_input = {
    "usernames": [
        "promodeamiga",
        "promosdarafa"
    ],
    "maxUsernames": 2,
    "delayBetweenRequests": 2000,
}

# Run the Actor and wait for it to finish
run = client.actor("7eNb0Yr5dLwy1jfPS").call(run_input=run_input)

# # Fetch and print Actor results from the run's dataset (if there are any)
# for item in client.dataset(run["defaultDatasetId"]).iterate_items():
#     print(item)

[apify.instagram-story-viewer runId:fsyHUY5VvbQna8fqC] -> Status: RUNNING, Message: 
[apify.instagram-story-viewer runId:fsyHUY5VvbQna8fqC] -> 2025-11-07T12:53:56.983Z ACTOR: Pulling container image of build 6SdYWbsdK1dAUtt2P from registry.
[apify.instagram-story-viewer runId:fsyHUY5VvbQna8fqC] -> 2025-11-07T12:53:56.985Z ACTOR: Creating container.
[apify.instagram-story-viewer runId:fsyHUY5VvbQna8fqC] -> 2025-11-07T12:53:57.034Z ACTOR: Starting container.
[apify.instagram-story-viewer runId:fsyHUY5VvbQna8fqC] -> 2025-11-07T12:53:57.873Z INFO  System info {"apifyVersion":"3.5.0","apifyClientVersion":"2.17.0","crawleeVersion":"3.15.1","osType":"Linux","nodeVersion":"v20.19.5"}
[apify.instagram-story-viewer runId:fsyHUY5VvbQna8fqC] -> 2025-11-07T12:53:59.396Z ✅ Stories checked for @promodeamiga → stories saved
[apify.instagram-story-viewer runId:fsyHUY5VvbQna8fqC] -> 2025-11-07T12:54:13.927Z ✅ Stories checked for @promosdarafa → stories saved
[apify.instagram-story-viewer runId:fsyHUY5Vv

In [12]:
import json

In [15]:
for item in client.dataset(run["defaultDatasetId"]).iterate_items():
    with open(f'output_{item["requested"]}.json', 'a', encoding='utf-8') as f:
        json.dump(item, f, ensure_ascii=False, indent=4)
        f.write('\n')

In [18]:
import duckdb as db
con = db.connect()
con.install_extension('json')
con.load_extension('json')

In [59]:
path = r'teste_json\*.json'
df = con.execute('''
with parte1 as (
SELECT
requested as username,
UNNEST(stories) as stories
FROM read_json_auto(?, ignore_errors=true)
),
parte2 as (
SELECT
username,
UNNEST(stories.story_link_stickers).story_link.display_url as story_link_url
FROM parte1
)   
SELECT username,
story_link_url,
case
when split(story_link_url, '/')[1] = 'amzlink.to' then 'Amazon'
when split(story_link_url, '/')[1] = 'mercadolivre.com' then 'Mercado Livre'
when split(story_link_url, '/')[1] = 's.shopee.com.br' then 'Shopee'
when split(story_link_url, '/')[1] = 'minhaloja.natura.com' then 'Natura'
when split(story_link_url, '/')[1] = 'magazinevoce.com.br' then 'Magazine Você'
when split(story_link_url, '/')[1] = 'elausa.com.br' then 'Ela Usa'
when split(story_link_url, '/')[1] = 'epocacosmeticos.com.br' then 'Época Cosméticos'
when split(story_link_url, '/')[1] = 'natura.com.br' then 'Natura'
when split(story_link_url, '/')[1] = 'sminhaloja.natura.com' then 'Natura'
else null end as origin
FROM parte2
''', [path]).df()
df

,username,story_link_url,origin
0,promodeamiga,amzlink.to/az05q8jo0se9d,Amazon
1,promodeamiga,amzlink.to/az0tlcdztiftp,Amazon
2,promodeamiga,amzlink.to/az02w9ijtni97,Amazon
3,promodeamiga,amzlink.to/az05jsmpit2ja,Amazon
4,promodeamiga,amzlink.to/az0sdsq4o5tld,Amazon
...,...,...,...
168,promosdarafa,s.shopee.com.br/6kwr3atvlu,Shopee
169,promosdarafa,mercadolivre.com/sec/2kwmeul,Mercado Livre
170,promosdarafa,sminhaloja.natura.com/wczwbj3,Natura
171,promosdarafa,amzlink.to/az0it4ahkek8d,Amazon
